In [1]:
pip install tensorflow-model-optimization

Note: you may need to restart the kernel to use updated packages.


In [24]:
import tensorflow as tf
import tensorflow_model_optimization as tfmot
import numpy as np
import keras
import os

from keras.datasets import mnist
from keras.utils import to_categorical

In [3]:
# Charger le modèle Keras à partir du fichier
model = tf.keras.models.load_model('mnist_model.h5')

# Afficher les détails du modèle
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 13, 13, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 5, 5, 64)         0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 1600)              0         
                                                                 
 dense (Dense)               (None, 64)                1

In [4]:
# Annoter les couches du modèle avec des informations de quantification
qat_model = tfmot.quantization.keras.quantize_annotate_model(model)

# Compiler le modèle
qat_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Afficher les détails du modèle
qat_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 quantize_annotate (Quantize  (None, 26, 26, 32)       320       
 Annotate)                                                       
                                                                 
 quantize_annotate_1 (Quanti  (None, 13, 13, 32)       0         
 zeAnnotate)                                                     
                                                                 
 quantize_annotate_2 (Quanti  (None, 11, 11, 64)       18496     
 zeAnnotate)                                                     
                                                                 
 quantize_annotate_3 (Quanti  (None, 5, 5, 64)         0         
 zeAnnotate)                                                     
                                                                 
 quantize_annotate_4 (Quanti  (None, 1600)             0

La fonction quantize_annotate_model() utilisée dans le code précédent ajoute des annotations de quantification aux couches du modèle, mais cela ne suffit pas à quantifier réellement le modèle. Les annotations de quantification sont simplement des marqueurs pour indiquer au modèle comment effectuer la quantification lors de l'entraînement.

In [5]:
# Charger les données MNIST
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

In [6]:
# Normaliser les données
x_train = x_train / 255.0
x_test = x_test / 255.0

In [7]:
# Réduire la taille du jeu de données
x_train = x_train[:10000]
y_train = y_train[:10000]
x_test = x_test[:2000]
y_test = y_test[:2000]

In [8]:
# Réentraîner le modèle sur un sous-ensemble des données
qat_model.fit(x_train, y_train, batch_size=32, epochs=2, validation_data=(x_test, y_test))

Epoch 1/2
313/313 [==============================] - 5s 12ms/step - loss: 0.0220 - accuracy: 0.9930 - val_loss: 0.0782 - val_accuracy: 0.9805
Epoch 2/2
313/313 [==============================] - 3s 11ms/step - loss: 0.0109 - accuracy: 0.9963 - val_loss: 0.0544 - val_accuracy: 0.9815


In [9]:
# Évaluer le modèle sur le jeu de données d'entraînement et de test
train_loss, train_acc = qat_model.evaluate(x_train, y_train, verbose=2)
test_loss, test_acc = qat_model.evaluate(x_test, y_test, verbose=2)
print(f'Train accuracy: {train_acc:.4f}, Test accuracy: {test_acc:.4f}')

313/313 - 1s - loss: 0.0030 - accuracy: 0.9997 - 1s/epoch - 4ms/step
63/63 - 0s - loss: 0.0544 - accuracy: 0.9815 - 249ms/epoch - 4ms/step
Train accuracy: 0.9997, Test accuracy: 0.9815


In [10]:
# Convertir le modèle TFLite
converter = tf.lite.TFLiteConverter.from_keras_model(qat_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model_quant = converter.convert()

# Sauvegarder le modèle TFLite
with open('qat_model.tflite', 'wb') as f:
    f.write(tflite_model_quant)

INFO:tensorflow:Assets written to: C:\Users\haddr\AppData\Local\Temp\tmp5s_7wy25\assets


INFO:tensorflow:Assets written to: C:\Users\haddr\AppData\Local\Temp\tmp5s_7wy25\assets


In [11]:
# Sauvegarder le modèle QAT
qat_model.save('qat_model.h5')

In [45]:
keras_model_size = os.path.getsize('mnist_model.h5')
print("Taille du modèle Keras : {:.2f} Ko".format(keras_model_size / 1024))

quantized_model_size = os.path.getsize('qat_model.tflite')
print("Taille du modèle quantifié : {:.2f} Ko".format(quantized_model_size / 1024))

qat_model_size = os.path.getsize('qat_model.h5')
print("Taille du modèle QAT : {:.2f} Ko".format(qat_model_size / 1024))

Taille du modèle Keras : 994.52 Ko
Taille du modèle quantifié : 126.74 Ko
Taille du modèle QAT : 1474.07 Ko


In [13]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

In [14]:
x_train = x_train / 255.0
x_test = x_test / 255.0

In [15]:
x_train = x_train[:10000]
y_train = y_train[:10000]
x_test = x_test[:2000]
y_test = y_test[:2000]

In [25]:
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images = train_images.reshape((60000, 28, 28, 1))
train_images = train_images.astype('float32') / 255

test_images = test_images.reshape((10000, 28, 28, 1))
test_images = test_images.astype('float32') / 255

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

In [38]:
mnist = keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

In [39]:
train_images = train_images / 255.0
test_images = test_images / 255.0

In [40]:
interpreter = tf.lite.Interpreter(model_path="model_optimized.tflite")
interpreter.allocate_tensors()
input_index = interpreter.get_input_details()[0]["index"]
output_index = interpreter.get_output_details()[0]["index"]

In [34]:
test_loss, test_acc = model.evaluate(test_images, test_labels)
print('"Accuracy du modèle original : {:.4f}', test_acc)

313/313 [==============================] - 2s 7ms/step - loss: 0.0351 - accuracy: 0.9884
"Accuracy du modèle original : {:.4f} 0.9883999824523926


In [42]:
correct_predictions = 0
for i in range(len(test_images)):
    # Préparer l'image
    input_shape = interpreter.get_input_details()[0]['shape']
    input_data = np.expand_dims(test_images[i], axis=0)
    input_data = np.resize(input_data, input_shape).astype(np.float32)
    interpreter.set_tensor(input_index, input_data)


    # Récupérer la prédiction
    output = interpreter.tensor(output_index)
    prediction = np.argmax(output()[0])

    # Vérifier si la prédiction est correcte
    if prediction == test_labels[i]:
        correct_predictions += 1

accuracy = correct_predictions / len(test_images)
print("Accuracy:", accuracy)


Accuracy: 0.1135


In [43]:
# Évaluer le modèle sur le jeu de données d'entraînement et de test
test_loss, test_acc = qat_model.evaluate(x_test, y_test, verbose=2)
print(f'Test accuracy: {test_acc:.4f}')

63/63 - 0s - loss: 0.0544 - accuracy: 0.9815 - 213ms/epoch - 3ms/step
Test accuracy: 0.9815


In [47]:
original_model_size = os.path.getsize('mnist_model.h5') / 1024
print("Taille du modèle original : {:.2f} Ko".format(original_model_size))

quantized_model_size = os.path.getsize('model_optimized.tflite') / 1024
print("Taille du modèle quantifié avec la post training quantization : {:.2f} Ko".format(quantized_model_size))

qat_model_size = os.path.getsize('qat_model.h5') / 1024
print("Taille du modèle entraîné avec la training aware quantization : {:.2f} Ko".format(qat_model_size))

Taille du modèle original : 994.52 Ko
Taille du modèle quantifié avec la post training quantization : 126.45 Ko
Taille du modèle entraîné avec la training aware quantization : 1474.07 Ko
